<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Library
## Example: STILT: footprints and timeseries

This example shows how to load timeseries data and footprints, and make some plots using Holoviews and Geoviews to create a map.

## Documentation
Full documentation for the library on the [project page](https://icos-carbon-portal.github.io/pylib/), how to install and wheel on [pypi.org](https://pypi.org/project/icoscp/), source is available on [github](https://github.com/ICOS-Carbon-Portal/pylib)

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib widget

import pandas as pd

#Import STILT tools:
from icoscp.stilt import stiltstation

### Create a STILT station object

In [ ]:
st = stiltstation.get(id='kit100')
print(st)

### Get time series data

In [ ]:
start = '2018-01-01'
end = '2018-12-31'

data = st.get_ts(start, end)
data.head()

### Plot STILT time series

In [ ]:
data.plot(y=['co2.stilt', 'co2.background'], title=st.id, ylabel='ppm', figsize=(8,4))

## Extract time series with columns = 'co2'
see documentation what columns you can return
[https://icos-carbon-portal.github.io/pylib/modules/#get_tsstart_date-end_date-hours-columns](https://icos-carbon-portal.github.io/pylib/modules/#get_tsstart_date-end_date-hours-columns)

In [ ]:
# set date constraints for the rest of this example notebook
start = '2018-01-01'
end = '2018-01-31'

In [ ]:
stiltdata = st.get_ts(start, end, columns='co2')
stiltdata.head()

In [ ]:
stiltdata.columns

In [ ]:
stiltdata.columns[2:13].to_list()

### Create a plot with all co2 components

In [ ]:
ax = stiltdata[stiltdata.columns[2:13].to_list()].plot()
ax.legend(loc='best', fontsize=8)
ax.plot()

### Aggregate by day
and create a stacked bar graph

In [ ]:
day = stiltdata.resample('D').sum()

In [ ]:
# plot the bar graph
ax1 = day[stiltdata.columns[2:13].to_list()].plot.bar(stacked='True')
ax1.legend(loc='best', fontsize=8)

# adjust the xticks
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

# display
ax1.plot()

### Aggregate by component

In [ ]:
component = stiltdata.agg('sum')

In [ ]:
fig_pie, ax_pie = plt.subplots()
fig_pie.suptitle('STILT components')
component.iloc[2:13].plot.pie(ax=ax_pie)
ax_pie.set_ylabel('')
ax_pie.set_xlabel('')
plt.show()

## Load observation and compare to model result

In [ ]:
from icoscp.cpb.dobj import Dobj

In [ ]:
kit100 = Dobj('https://hdl.handle.net/11676/LJ4uetvEho7-k9K9TUnLHfFh')

In [ ]:
kit100.citation

### create a mask to get the same timeframe

In [ ]:
# we have set start and end date above (cell number 5)
mask = (kit100.data['TIMESTAMP'] >= start) & (kit100.data['TIMESTAMP'] <= end)
obsdata = kit100.data[mask]
obsdata.set_index('TIMESTAMP', inplace=True)
obsdata['co2']

### Resample STILT data
Because the observation are hourly aggregates, we resample the STILT output to make our lives easier to compare the observation vs model.

In [ ]:
stilthourly = stiltdata.resample('1H').mean().interpolate('linear')
stilthourly['co2.stilt']

### Data harmonization and plot
If you look at the lenght of the dataframes above you will see a discrepancy. Observation contains less data points. Most likely an interuption of the measurement or QA/QC removed values. We need to merge the files together on index (both have now the Time/Date as index). Missing values will be NaN in the pandas data frame

In [ ]:
harmonized = stilthourly.join(obsdata)
harmonized.plot(y = ['co2.stilt', 'co2'], use_index=True, grid=True, linewidth=0.5)

### Plot difference

In [ ]:
harmonized['diff'] = harmonized['co2.stilt']-harmonized['co2']

In [ ]:
fig_dif, ax_dif = plt.subplots()
ax_dif = plt.axes()
ax_dif.plot(harmonized['diff'])
ax_dif.grid(color='0.9')
ax_dif.set_ylabel('diff (ppm)')

# adjust the xticks
ax_dif.xaxis.set_major_formatter(mdates.DateFormatter('%d'))